# INTRODUCTION

In this exercise, you will assist a company that sells IT accessories in answering some business questions using data and visuals, thereby contributing to enhancing their decision-making processes quickly and effectively.

Explore the provided invoices data at each sales point alongside other company's files and answer the following questions. **You are encouraged to use only the method/approaches discussed in our class where possible.**

# With a help of an appropriate graph, Illustrate and comment on the distribution of total products sold at each sales point?

In [1]:
# Your Answer

# Effectively display the revenue generated by each sales officer over time. Explain any quick insights from the resulting graph.

In [2]:
# Your Answer

# Determine the most active days of the week for each product category by analyzing the average number of products sold and display the result using an appropriate graph.  What are your thoughts on the resulting graph?

In [3]:
# Your Answer

# Reshape the data to display the total revenue amount of each product by year side by side. Dispaly the resulting dataframe.

In [4]:
# Your Answer